In [1]:
import os
import random
from tqdm import tqdm
import torch

print(torch.__version__)

1.0.1.post2


In [82]:
# consts
split_name = 'voc-1'
root_path = '/home/voyager/data/root/voc/'
train_ratio = 0.8

image_path = os.path.join(root_path, 'JPEGImages')
anno_path = os.path.join(root_path, 'Annotations')
split_path = os.path.join(root_path, 'ImageSets/Main')

# list annotation
anno_file_list = os.listdir(anno_path)
anno_list = [anno_file.split('.')[0] for anno_file in anno_file_list]

# sample
train_list = random.sample(anno_list, int(len(anno_list) * train_ratio))

val_list = []
for anno in anno_list:
    if anno not in train_list:
        val_list.append(anno)
        
# save
with open(os.path.join(split_path, '{}_{}.txt'.format(
    split_name,
    'train'
)), 'w') as train_file:
    for train_anno in train_list:
        train_file.write('{}\n'.format(train_anno))
        
with open(os.path.join(split_path, '{}_{}.txt'.format(
    split_name,
    'val'
)), 'w') as val_file:
    for val_anno in val_list:
        val_file.write('{}\n'.format(val_anno))
        

## some dirty work

In [68]:
import PIL.Image
import uuid

# convert to jpg
def convert_imgs(img_path):
    img_list = os.listdir(img_path)
    
    for filename in tqdm(img_list):
        arr = filename.split('.')

        im = PIL.Image.open(os.path.join(img_path, filename))
        
        # we have read image, remove the original one
        os.remove(os.path.join(img_path, filename))
        
        im = im.convert('RGB')
        im.save(os.path.join(img_path, '{}.jpg'.format(arr[0])))
        
# rename with uuid
def convert_names(root_path, no):
    img_path = os.path.join(root_path, 'img_{}/'.format(no))
    anno_path = os.path.join(root_path, 'anno_{}/'.format(no))
    
    img_list = os.listdir(img_path)
    anno_list = os.listdir(anno_path)
    
    table = []
    
    for anno_file in tqdm(anno_list):
        arr = anno_file.split('.')
        name = arr[0]
        
        if len(name.split('-')) > 1:
            score = int(name.split('-')[1])
        elif len(name.split('(')) > 1:
            score = name.split('(')[1]
            score = int(score.split(')')[0])
        elif len(name.split('（')) > 1:
            score = name.split('（')[1]
            score = int(score.split('）')[0])
        else:
            print(name)
            
        uid = uuid.uuid4()
        
        os.rename(
            os.path.join(img_path, '{}.jpg'.format(name)),
            os.path.join(img_path, '{}.jpg'.format(uid, score))
        )
        
        os.rename(
            os.path.join(anno_path, '{}.xml'.format(name)),
            os.path.join(anno_path, '{}.xml'.format(uid))
        )
        
        table.append({
            'uid': uid,
            'score': score
        })
        
    return table

dirty_root_path = '/home/voyager/data/root/dirty/'


In [72]:
img1_path = os.path.join(dirty_root_path, 'img_1')
convert_imgs(img1_path)

100%|██████████| 1137/1137 [00:12<00:00, 90.23it/s]


In [73]:
img2_path = os.path.join(dirty_root_path, 'img_2')
convert_imgs(img2_path)

100%|██████████| 447/447 [00:06<00:00, 70.72it/s]


In [78]:
img3_path = os.path.join(dirty_root_path, 'img_3')
convert_imgs(img3_path)

100%|██████████| 39/39 [00:00<00:00, 79.31it/s]


please remove non-jpeg files under image folders

there's a sample called 33（73), please change both jpg and xml to 33(73) by hand

In [74]:
table_1 = convert_names(dirty_root_path, 1)

100%|██████████| 1111/1111 [00:00<00:00, 19156.80it/s]


In [75]:
table_2 = convert_names(dirty_root_path, 2)

100%|██████████| 434/434 [00:00<00:00, 11176.71it/s]


In [79]:
table_3 = convert_names(dirty_root_path, 3)

100%|██████████| 39/39 [00:00<00:00, 7487.09it/s]


now, export score list

In [80]:
table = table_1.copy()
table.extend(table_2)
table.extend(table_3)

score_path = os.path.join(root_path, 'score.csv')
print(score_path)

with open(score_path, 'w') as score_csv:
    score_csv.write('id,score\n')
    
    for score in table:
        score_csv.write('{},{}\n'.format(score['uid'], score['score']))


/home/voyager/data/root/score.csv


Now copy images to JPEGImages/, annotations to Annotations/

let's try to read with voc dataset

In [1]:
from dataset import voc
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as trans
import matplotlib.pyplot as plt
import numpy as np

# consts
split_name = 'voc-1'
root_path = '/home/voyager/data/root/voc/'

train_trans = trans.Compose([
    trans.Resize((600, 600)),
    trans.ToTensor()
])

# train set
train_set = voc.VOCDetection(
    root_path,
    image_set="voc-1_train",
    transforms=train_trans
)

train_loader = DataLoader(
    train_set,
    batch_size=4,
    shuffle=True,
    collate_fn=voc.collate
)

for i, batch in enumerate(train_loader):
    imgs, annos = batch
    
    img = imgs[0].numpy()
    img = np.transpose(img, (1, 2, 0))
    
    plt.imshow(img)
    
    break
    

anno: (tensor([0, 1]), tensor([[364., 167., 487., 402.],
        [350.,  85., 488., 401.]]))
anno: (tensor([0, 1]), tensor([[257., 225., 334., 384.],
        [203., 226., 335., 556.]]))
anno: (tensor([0, 1]), tensor([[ 76., 128., 222., 394.],
        [ 77.,   1., 257., 396.]]))
anno: (tensor([0, 1]), tensor([[522.,  83., 738., 495.],
        [523.,  85., 820., 745.]]))
[(tensor([0, 1]), tensor([[364., 167., 487., 402.],
        [350.,  85., 488., 401.]])), (tensor([0, 1]), tensor([[257., 225., 334., 384.],
        [203., 226., 335., 556.]])), (tensor([0, 1]), tensor([[ 76., 128., 222., 394.],
        [ 77.,   1., 257., 396.]])), (tensor([0, 1]), tensor([[522.,  83., 738., 495.],
        [523.,  85., 820., 745.]]))]
